<a href="https://colab.research.google.com/github/taniajasmin/Machine-Learning-Basic-Projects/blob/main/MovieRecommendationSystem_Description.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import kagglehub

# path = kagglehub.dataset_download("carolzhangdc/imdb-5000-movie-dataset")
# print("Path to dataset files:", path)

100%|██████████| 554k/554k [00:00<00:00, 695kB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/carolzhangdc/imdb-5000-movie-dataset/versions/1


In [2]:
# import os, re
# import pandas as pd
# import numpy as np

# # Find CSV
# candidates = [f for f in os.listdir(path) if f.lower().endswith(".csv")]
# print("Found CSV files:", candidates)
# csv_path = os.path.join(path, "movie_metadata.csv") if "movie_metadata.csv" in candidates else os.path.join(path, candidates[0])

# df = pd.read_csv(csv_path, encoding="latin-1")

# # Basic tidy columns we'll use
# df["title"] = df["movie_title"].astype(str).str.strip()
# for c in ["genres","plot_keywords","director_name","actor_1_name","actor_2_name","actor_3_name","language","country"]:
#     if c not in df.columns: df[c] = ""
#     df[c] = df[c].fillna("").astype(str)

# # Keep only rows that have a non-empty title and at least one of the text fields
# mask = (df["title"].str.len()>0) & (
#     df["genres"].str.len()+df["plot_keywords"].str.len()+df["director_name"].str.len()+
#     df["actor_1_name"].str.len()+df["actor_2_name"].str.len()+df["actor_3_name"].str.len() > 0
# )
# df = df.loc[mask].drop_duplicates(subset="title").reset_index(drop=True)
# print("Movies kept:", len(df))
# df[["title","genres","plot_keywords","director_name","actor_1_name"]].head()

Found CSV files: ['movie_metadata.csv']
Movies kept: 4916


,title,genres,plot_keywords,director_name,actor_1_name
0,AvatarÂ,Action|Adventure|Fantasy|Sci-Fi,avatar|future|marine|native|paraplegic,James Cameron,CCH Pounder
1,Pirates of the Caribbean: At World's EndÂ,Action|Adventure|Fantasy,goddess|marriage ceremony|marriage proposal|pi...,Gore Verbinski,Johnny Depp
2,SpectreÂ,Action|Adventure|Thriller,bomb|espionage|sequel|spy|terrorist,Sam Mendes,Christoph Waltz
3,The Dark Knight RisesÂ,Action|Thriller,deception|imprisonment|lawlessness|police offi...,Christopher Nolan,Tom Hardy
4,Star Wars: Episode VII - The Force AwakensÂ,Documentary,,Doug Walker,Doug Walker


In [3]:
df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,title
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000,AvatarÂ
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0,Pirates of the Caribbean: At World's EndÂ
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000,SpectreÂ
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000,The Dark Knight RisesÂ
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,,,NaN,NaN,NaN,12.0,7.1,NaN,0,Star Wars: Episode VII - The Force AwakensÂ


In [10]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("prajitdatta/movielens-100k-dataset")

print("Path to dataset files:", path)

100%|██████████| 4.77M/4.77M [00:01<00:00, 3.50MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/prajitdatta/movielens-100k-dataset/versions/1


In [13]:
import os, pandas as pd, numpy as np

def find_file(root, candidates):
    for dp, _, files in os.walk(root):
        for f in files:
            if f in candidates:
                return os.path.join(dp, f)
    return None

ratings_fp = find_file(path, ["u.data", "ratings.csv"])
movies_fp  = find_file(path, ["u.item", "movies.csv"])
print("ratings:", ratings_fp, "\nmovies:", movies_fp)

# --- read ratings
if ratings_fp.endswith("u.data"):
    ratings = pd.read_csv(
        ratings_fp, sep="\t",
        names=["user_id","item_id","rating","timestamp"],
        engine="python"
    )
else:  # ratings.csv style
    r = pd.read_csv(ratings_fp)
    # normalize column names
    cols = {c.lower(): c for c in r.columns}
    ratings = r.rename(columns={
        cols.get("userid","userId"): "user_id",
        cols.get("movieid","movieId"): "item_id",
        cols.get("rating"): "rating",
        cols.get("timestamp","Timestamp"): "timestamp",
    })[["user_id","item_id","rating","timestamp"]]

# --- read movies
if movies_fp.endswith("u.item"):
    movies = pd.read_csv(
        movies_fp, sep="|", usecols=[0,1],
        names=["item_id","title"], encoding="latin-1", engine="python"
    )
else:  # movies.csv style
    m = pd.read_csv(movies_fp, encoding="latin-1")
    cols = {c.lower(): c for c in m.columns}
    movies = m.rename(columns={
        cols.get("movieid","movieId"): "item_id",
        cols.get("title"): "title",
    })[["item_id","title"]]

print(ratings.shape, "ratings; ", movies.shape, "movies")

ratings: /root/.cache/kagglehub/datasets/prajitdatta/movielens-100k-dataset/versions/1/ml-100k/u.data 
movies: /root/.cache/kagglehub/datasets/prajitdatta/movielens-100k-dataset/versions/1/ml-100k/u.item
(100000, 4) ratings;  (1682, 2) movies


In [12]:
ratings.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


Building the user–item matrix

In [14]:
ui = ratings.pivot_table(index="user_id", columns="item_id", values="rating").astype(float)
user_ids = ui.index.values
item_ids = ui.columns.values
titles = movies.set_index("item_id").title
ui.shape

(943, 1682)

In [15]:
R = ui.values.copy()
mask = ~np.isnan(R)

rng = np.random.default_rng(42)
test_mask = np.zeros_like(R, dtype=bool)

for u in range(R.shape[0]):
    idx = np.where(mask[u])[0]
    if idx.size >= 5:
        k = max(1, int(0.2 * idx.size))
        chosen = rng.choice(idx, size=k, replace=False)
        test_mask[u, chosen] = True

R_train = R.copy(); R_train[test_mask] = np.nan
R_test  = np.full_like(R, np.nan); R_test[test_mask] = R[test_mask]

np.isfinite(R_train).sum(), np.isfinite(R_test).sum()

(np.int64(80367), np.int64(19633))

In [16]:
R = ui.values.copy()
mask = ~np.isnan(R)

rng = np.random.default_rng(42)
test_mask = np.zeros_like(R, dtype=bool)

for u in range(R.shape[0]):
    idx = np.where(mask[u])[0]
    if idx.size >= 5:
        k = max(1, int(0.2 * idx.size))
        chosen = rng.choice(idx, size=k, replace=False)
        test_mask[u, chosen] = True

R_train = R.copy(); R_train[test_mask] = np.nan
R_test  = np.full_like(R, np.nan); R_test[test_mask] = R[test_mask]

np.isfinite(R_train).sum(), np.isfinite(R_test).sum()

(np.int64(80367), np.int64(19633))

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def predict_user_user(R_train, k_neighbors=40):
    means = np.nanmean(R_train, axis=1, keepdims=True)
    X = np.nan_to_num(R_train - means, nan=0.0)  # center and fill NaN with 0
    S = cosine_similarity(X)                     # user x user
    np.fill_diagonal(S, 0.0)

    # keep top-K neighbors per user
    if k_neighbors:
        kth = min(k_neighbors, S.shape[1]-1) - 1
        idx = np.argpartition(-np.abs(S), kth=kth, axis=1)
        mask = np.zeros_like(S, dtype=bool)
        rows = np.arange(S.shape[0])[:, None]
        mask[rows, idx[:, :k_neighbors]] = True
        S = S * mask

    denom = np.clip(np.abs(S).sum(axis=1, keepdims=True), 1e-8, None)
    P_centered = (S @ X) / denom
    return P_centered + means

P_user = predict_user_user(R_train, k_neighbors=40)
P_user.shape

(943, 1682)

In [18]:
def precision_at_k(P, R_train, R_test, k=10, rel_threshold=4.0):
    U, I = P.shape
    train_seen = np.isfinite(R_train)
    test_rel   = (R_test >= rel_threshold)

    precisions = []
    for u in range(U):
        candidates = np.where(~train_seen[u])[0]
        if candidates.size == 0:
            continue
        top = candidates[np.argsort(P[u, candidates])[::-1][:k]]
        hits = np.intersect1d(top, np.where(test_rel[u])[0]).size
        precisions.append(hits / k)
    return float(np.mean(precisions)), len(precisions)

p10_user, n_eval = precision_at_k(P_user, R_train, R_test, k=10, rel_threshold=4.0)
print(f"User-CF Precision@10: {p10_user:.3f}  (users evaluated: {n_eval})")

User-CF Precision@10: 0.177  (users evaluated: 943)


In [19]:
def recommend_for_user(user_id, P, R_train, top_n=10):
    uidx = np.where(user_ids == user_id)[0][0]
    seen = np.isfinite(R_train[uidx])
    scores = P[uidx].copy()
    scores[seen] = -np.inf
    order = np.argsort(scores)[::-1][:top_n]
    mids = item_ids[order]
    return pd.DataFrame({
        "rank": np.arange(1, top_n+1),
        "movie_id": mids,
        "title": titles.reindex(mids).values,
        "pred_score": scores[order]
    })

recommend_for_user(int(user_ids[0]), P_user, R_train, top_n=10)

,rank,movie_id,title,pred_score
0,1,98,"Silence of the Lambs, The (1991)",4.196522
1,2,168,Monty Python and the Holy Grail (1974),4.158888
2,3,127,"Godfather, The (1972)",4.092507
3,4,89,Blade Runner (1982),4.061720
4,5,318,Schindler's List (1993),4.016842
5,6,183,Alien (1979),4.003597
6,7,22,Braveheart (1995),3.988076
7,8,313,Titanic (1997),3.979265
8,9,603,Rear Window (1954),3.923992
9,10,182,GoodFellas (1990),3.891754


Item-based CF + SVD

In [20]:
from sklearn.decomposition import TruncatedSVD

def predict_item_item(R_train, k_neighbors=80):
    means = np.nanmean(R_train, axis=1, keepdims=True)
    X = np.nan_to_num(R_train - means, nan=0.0)  # U x I
    S = cosine_similarity(X.T)        # I x I
    np.fill_diagonal(S, 0.0)
    if k_neighbors:
        kth = min(k_neighbors, S.shape[1]-1) - 1
        idx = np.argpartition(-np.abs(S), kth=kth, axis=1)
        mask = np.zeros_like(S, dtype=bool)
        rows = np.arange(S.shape[0])[:, None]
        mask[rows, idx[:, :k_neighbors]] = True
        S = S * mask
    denom = np.clip(np.abs(S).sum(axis=0, keepdims=True), 1e-8, None)
    P_centered = (X @ S) / denom
    return P_centered + means

def predict_svd(R_train, n_components=50, random_state=42):
    X = np.nan_to_num(R_train, nan=0.0)
    svd = TruncatedSVD(n_components=n_components, random_state=random_state)
    U_sigma = svd.fit_transform(X)
    Vt = svd.components_
    return U_sigma @ Vt

P_item = predict_item_item(R_train, k_neighbors=80)
p10_item, _ = precision_at_k(P_item, R_train, R_test, k=10)
print(f"Item-CF Precision@10: {p10_item:.3f}")

P_svd = predict_svd(R_train, n_components=50)
p10_svd, _ = precision_at_k(P_svd, R_train, R_test, k=10)
print(f"SVD (rank=50) Precision@10: {p10_svd:.3f}")

Item-CF Precision@10: 0.147
SVD (rank=50) Precision@10: 0.210


Popularity baseline (non-personalized) + metric

In [21]:
# Top-N by global average rating (min count filter)
pop_stats = ratings.groupby("item_id").agg(avg=("rating","mean"), cnt=("rating","count"))
pop_stats = pop_stats[pop_stats["cnt"] >= 20].sort_values("avg", ascending=False)

In [22]:
def popularity_precision_at_k(R_train, R_test, k=10, rel_threshold=4.0):
    top_items = pop_stats.index.values[:k]
    test_rel = (R_test >= rel_threshold)
    precisions = []
    for u in range(R_test.shape[0]):
        # exclude items the user already saw in TRAIN
        seen = np.isfinite(R_train[u])
        recs = [i for i in top_items if not seen[np.where(item_ids==i)[0][0]]][:k]
        # map recs to column indices
        idx = [np.where(item_ids==i)[0][0] for i in recs]
        hits = np.isfinite(R_test[u, idx]) & (R_test[u, idx] >= rel_threshold)
        precisions.append(hits.sum()/k if len(idx)==k else 0.0)
    return float(np.mean(precisions))

p10_pop = popularity_precision_at_k(R_train, R_test, k=10, rel_threshold=4.0)
print(f"Popularity baseline Precision@10: {p10_pop:.3f}")

Popularity baseline Precision@10: 0.005


Metrics: Recall@K, MAP@K, nDCG@K

In [26]:
def eval_at_k(P, R_train, R_test, k=10, rel_threshold=4.0):
    U, I = P.shape
    train_seen = np.isfinite(R_train)
    test_rel = (R_test >= rel_threshold)
    precs, recs, maps, ndcgs = [], [], [], []

    for u in range(U):
        rel_idx = np.where(test_rel[u])[0]
        if rel_idx.size == 0:
            continue
        candidates = np.where(~train_seen[u])[0]
        if candidates.size == 0:
            continue
        scores = P[u, candidates]
        order = candidates[np.argsort(scores)[::-1][:k]]
        hits = np.isin(order, rel_idx)

        # Precision@K
        precs.append(hits.sum()/k)

        # Recall@K
        recs.append(hits.sum()/rel_idx.size)

        # AP@K
        if hits.any():
            cum_hits = np.cumsum(hits)
            precisions = cum_hits[hits] / (np.nonzero(hits)[0]+1)
            maps.append(precisions.mean())
        else:
            maps.append(0.0)

        # nDCG@K (binary relevance)
        gains = hits.astype(float) / np.log2(np.arange(2, k+2))
        dcg = gains.sum()
        ideal = np.ones(min(k, rel_idx.size)) / np.log2(np.arange(2, min(k, rel_idx.size)+2))
        idcg = ideal.sum()
        ndcgs.append(dcg / idcg if idcg > 0 else 0.0)

    return {
        "users_eval": len(precs),
        "P@K": float(np.mean(precs)) if precs else 0.0,
        "R@K": float(np.mean(recs)) if recs else 0.0,
        "MAP@K": float(np.mean(maps)) if maps else 0.0,
        "nDCG@K": float(np.mean(ndcgs)) if ndcgs else 0.0,
    }

metrics_user = eval_at_k(P_user, R_train, R_test, k=10)
metrics_user

{'users_eval': 929,
 'P@K': 0.1797631862217438,
 'R@K': 0.17489382755211905,
 'MAP@K': 0.3823922688917691,
 'nDCG@K': 0.24451004264228135}